In [1]:
from google.colab import files
uploades = files.upload()

Saving peliculas_enriquecidas.csv to peliculas_enriquecidas.csv


In [29]:
import pandas as pd

df = pd.read_csv('peliculas_enriquecidas.csv')
df = df.dropna(subset=['overview_lemmatized', 'title'])
df = df.reset_index(drop=True)
df.head()

,id,title,original_title,production_countries,overview,release_date,original_language,genres,runtime,budget,...,cast,vote_average,director,keywords,main_country,overview_tokens,overview_lemmas,overview_lemmatized,keywords_tfidf,topic_id
0,10112,Los aristogatos,The Aristocats,['United States of America'],1910. En París una anciana millonaria y excént...,1970-12-24,en,"['Animación', 'Comedia', 'Familia', 'Aventura']",78,4000000,...,"['Phil Harris', 'Eva Gabor', 'Sterling Holloway']",7.287,Wolfgang Reitherman,"['paris, france', 'return', 'butler', 'cartoon...",United States of America,"['parís', 'anciana', 'millonaria', 'excéntrica...","['parís', 'anciana', 'millonario', 'excéntrico...",parís anciana millonario excéntrico llevar vid...,"['gato', 'enorme', 'único', 'ayuda', 'parís']",1
1,1685,Regreso al planeta de los simios,Beneath the Planet of the Apes,['United States of America'],Una nueva nave procedente del espacio y del ti...,1970-04-23,en,"['Aventura', 'Ciencia ficción', 'Misterio']",95,3000000,...,"['James Franciscus', 'Kim Hunter', 'Maurice Ev...",6.147,Ted Post,"['dystopia', 'mutant', 'distant future', 'astr...",United States of America,"['nueva', 'nave', 'procedente', 'espacio', 'ti...","['nuevo', 'nave', 'procedente', 'espacio', 'ti...",nuevo nave procedente espacio tiempo aterriza ...,"['humano', 'dominar', 'metro', 'prohibido', 'm...",7
2,9062,Love Story,Love Story,['United States of America'],Historia de amor de dos universitarios de Harv...,1970-12-16,en,"['Romance', 'Drama']",99,2200000,...,"['Ali MacGraw', ""Ryan O'Neal"", 'John Marley']",6.821,Arthur Hiller,"['based on novel or book', 'harvard university...",United States of America,"['historia', 'amor', 'dos', 'universitarios', ...","['historia', 'amor', 'dos', 'universitario', '...",historia amor dos universitario harvard perten...,"['social', 'distinto', 'universitario', 'difer...",6
3,11202,Patton,Patton,['United States of America'],Biografía del general norteamericano George S....,1970-01-25,en,"['Bélica', 'Drama', 'Historia']",172,12000000,...,"['George C. Scott', 'Stephen Young', 'Frank La...",7.487,Franklin J. Schaffner,"['general', 'steel helmet', 'allies', 'world w...",United States of America,"['biografía', 'general', 'norteamericano', 'ge...","['biografía', 'general', 'norteamericano', 'ge...",biografía general norteamericano george patton...,"['vencer', 'conducir', 'norte', 'tropa', 'áfri...",5
4,42589,"Hola, mamá","Hi, Mom!",['United States of America'],"John Rubin, un veterano de la guerra del Vietn...",1970-04-27,en,"['Comedia', 'Crimen']",88,0,...,"['Robert De Niro', 'Jennifer Salt', 'Allen Gar...",6.293,Brian De Palma,"['new york city', 'pornography', 'black panthe...",United States of America,"['john', 'rubin', 'veterano', 'guerra', 'vietn...","['john', 'rubin', 'veterano', 'guerra', 'vietn...",john rubin veterano guerra vietnam vuelve nuev...,"['apartamento', 'vietnam', 'asunto', 'serio', ...",7


In [30]:
# Vectorizamos cada campo por separado

from sklearn.feature_extraction.text import TfidfVectorizer

In [31]:
# Overview

tfidf_overview = TfidfVectorizer(max_features=5000)
X_overview = tfidf_overview.fit_transform(df['overview_lemmatized'].astype(str))

In [32]:
# keywords

tfidf_keywords = TfidfVectorizer()
X_keywords = tfidf_keywords.fit_transform(df['keywords_tfidf'].astype(str))

In [33]:
# Genres

tfidf_genres = TfidfVectorizer()
X_genres = tfidf_genres.fit_transform(df['genres'].astype(str))

In [34]:
# Director

tfidf_director = TfidfVectorizer()
X_director = tfidf_director.fit_transform(df['director'].astype(str))

In [35]:
# TopicID

df['topic_id_str'] = df['topic_id'].apply(lambda x: f"topic_{int(x)}")
tfidf_topic = TfidfVectorizer()
X_topic = tfidf_topic.fit_transform(df['topic_id_str'])

In [36]:
# País de producción

tfidf_country = TfidfVectorizer()
X_country = tfidf_country.fit_transform(df['production_countries'].astype(str))

In [37]:
# Normalizamos y combinamos matrices TF-IDF con pesos

from sklearn.preprocessing import normalize
from scipy.sparse import hstack

# Normalizamos
X_overview_n = normalize(X_overview)
X_keywords_n = normalize(X_keywords)
X_genres_n = normalize(X_genres)
X_director_n = normalize(X_director)
X_country_n = normalize(X_country)
X_topic_n = normalize(X_topic)

# Aplicamos pesos
X_overview_w = 0.4 * X_overview_n
X_keywords_w = 0.2 * X_keywords_n
X_genres_w = 0.15 * X_genres_n
X_director_w = 0.1 * X_director_n
X_country_w = 0.05 * X_country_n
X_topic_w = 0.1 * X_topic_n

# Concatenamos horizontalmente
X_combined = hstack([
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
])

In [38]:
# Calculamos la similitud

from sklearn.metrics.pairwise import cosine_similarity

similarity_matrix = cosine_similarity(X_combined)

In [39]:
# Función de recomendación

def recomendar_peliculas(titulo_pelicula, df, X_overview_w, X_keywords_w, X_genres_w, X_director_w, X_country_w, X_topic_w, top_n=5):
  if titulo_pelicula not in df['title'].values:
    print("Pelicula no encontrada.")
    return

  idx = df[df['title'] == titulo_pelicula].index[0]

  # Calcular similitudes individuales
  sim_overview = cosine_similarity(X_overview_w[idx], X_overview_w).flatten()
  sim_keywords = cosine_similarity(X_keywords_w[idx], X_keywords_w).flatten()
  sim_genres = cosine_similarity(X_genres_w[idx], X_genres_w).flatten()
  sim_director = cosine_similarity(X_director_w[idx], X_director_w).flatten()
  sim_country = cosine_similarity(X_country_w[idx], X_country_w).flatten()
  sim_topic = cosine_similarity(X_topic_w[idx], X_topic_w).flatten()

  # Sumamos similitudes ponderadas
  sim_total = sim_overview + sim_keywords + sim_genres + sim_director + sim_country + sim_topic

  #Creamos dataframe temporal con los resultados
  df_sim = df.copy()
  df_sim['similitud'] = sim_total

  # Eliminamos película base de los resultados
  df_sim = df_sim.drop(idx)

  # Obtenemos recomendaciones
  recomendaciones = df_sim.sort_values(by='similitud', ascending=False).head(top_n)

  print(f"\nPelícula base: {titulo_pelicula}\nRecomendaciones:\n")

  for i, row in recomendaciones.iterrows():
    contribuciones = {
        'overview': sim_overview[i],
        'keywords': sim_keywords[i],
        'genres': sim_genres[i],
        'director': sim_director[i],
        'country': sim_country[i],
        'topic': sim_topic[i]

    }

    razones = sorted(contribuciones.items(), key=lambda x: x[1], reverse=True)
    razones_utiles = [r[0] for r in razones if r[1] > 0.01][:2]

    print(f"🎬 {row['title']} (ID: {row['id']}) - Similitud: {row['similitud']:.3f} - Razones: {razones_utiles}")




In [40]:
# Prueba

recomendar_peliculas(
    'Constantine',
    df,
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
)


Película base: Constantine
Recomendaciones:

🎬 El gato infernal (ID: 20701) - Similitud: 2.573 - Razones: ['topic', 'genres']
🎬 Hostage (ID: 2026) - Similitud: 2.319 - Razones: ['country', 'topic']
🎬 Crash (Colisión) (ID: 1640) - Similitud: 2.257 - Razones: ['country', 'topic']
🎬 Vidas al límite (ID: 7873) - Similitud: 2.115 - Razones: ['topic', 'country']
🎬 Delta Force 2 (ID: 19086) - Similitud: 2.083 - Razones: ['topic', 'country']


In [41]:
recomendar_peliculas(
    'Los aristogatos',
    df,
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
)


Película base: Los aristogatos
Recomendaciones:

🎬 Kung Fu Panda 4 (ID: 1011985) - Similitud: 2.829 - Razones: ['country', 'topic']
🎬 Garfield: La película (ID: 748783) - Similitud: 2.784 - Razones: ['topic', 'genres']
🎬 Valiant (ID: 14175) - Similitud: 2.558 - Razones: ['topic', 'genres']
🎬 Vaiana 2 (ID: 1241982) - Similitud: 2.544 - Razones: ['topic', 'genres']
🎬 La isla (ID: 1635) - Similitud: 2.512 - Razones: ['country', 'topic']


In [42]:
recomendar_peliculas(
    'Delta Force 2',
    df,
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
)


Película base: Delta Force 2
Recomendaciones:

🎬 Hostage (ID: 2026) - Similitud: 2.667 - Razones: ['topic', 'country']
🎬 Wolfs (ID: 877817) - Similitud: 2.615 - Razones: ['country', 'topic']
🎬 Distrito 34: Corrupción total (ID: 31598) - Similitud: 2.554 - Razones: ['country', 'topic']
🎬 48 horas más (ID: 11595) - Similitud: 2.530 - Razones: ['country', 'topic']
🎬 Vidas al límite (ID: 7873) - Similitud: 2.521 - Razones: ['country', 'topic']


In [43]:
recomendar_peliculas(
    'Wolfs',
    df,
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
)


Película base: Wolfs
Recomendaciones:

🎬 48 horas más (ID: 11595) - Similitud: 2.863 - Razones: ['country', 'topic']
🎬 Ejecutivo ejecutor (ID: 20876) - Similitud: 2.847 - Razones: ['country', 'topic']
🎬 Bad Boys: Ride or Die (ID: 573435) - Similitud: 2.756 - Razones: ['country', 'topic']
🎬 El clan de los Rompehuesos (ID: 9291) - Similitud: 2.721 - Razones: ['country', 'topic']
🎬 Distrito 34: Corrupción total (ID: 31598) - Similitud: 2.696 - Razones: ['country', 'topic']


In [46]:
recomendar_peliculas(
    'Garfield: La película',
    df,
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
)


Película base: Garfield: La película
Recomendaciones:

🎬 Los aristogatos (ID: 10112) - Similitud: 2.784 - Razones: ['topic', 'genres']
🎬 Valiant (ID: 14175) - Similitud: 2.451 - Razones: ['topic', 'genres']
🎬 Vaiana 2 (ID: 1241982) - Similitud: 2.375 - Razones: ['topic', 'genres']
🎬 Chicken Little (ID: 9982) - Similitud: 2.370 - Razones: ['director', 'genres']
🎬 Kung Fu Panda 4 (ID: 1011985) - Similitud: 2.297 - Razones: ['topic', 'genres']


In [47]:
recomendar_peliculas(
    'Eva al desnudo',
    df,
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
)


Película base: Eva al desnudo
Recomendaciones:

🎬 Su propia vida (ID: 176734) - Similitud: 3.232 - Razones: ['genres', 'country']
🎬 La segunda mujer (ID: 62661) - Similitud: 3.014 - Razones: ['genres', 'country']
🎬 Rocky V (ID: 1375) - Similitud: 3.008 - Razones: ['genres', 'country']
🎬 El semental italiano (ID: 4255) - Similitud: 2.655 - Razones: ['country', 'topic']
🎬 Una vida por delante (ID: 1947) - Similitud: 2.610 - Razones: ['genres', 'topic']


In [48]:
recomendar_peliculas(
    'Patton',
    df,
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
)


Película base: Patton
Recomendaciones:

🎬 Bill, qué grande eres (ID: 54615) - Similitud: 2.587 - Razones: ['country', 'topic']
🎬 El reino de los cielos (ID: 1495) - Similitud: 2.320 - Razones: ['topic', 'genres']
🎬 Los chicos de la Nickel (ID: 1028196) - Similitud: 2.297 - Razones: ['country', 'topic']
🎬 Cerco de fuego (ID: 48897) - Similitud: 2.290 - Razones: ['country', 'topic']
🎬 Buenas noches, y buena suerte (ID: 3291) - Similitud: 2.279 - Razones: ['topic', 'genres']


In [49]:
recomendar_peliculas(
    'El especialista',
    df,
    X_overview_w,
    X_keywords_w,
    X_genres_w,
    X_director_w,
    X_country_w,
    X_topic_w
)


Película base: El especialista
Recomendaciones:

🎬 El padre de la novia (ID: 20758) - Similitud: 2.346 - Razones: ['topic', 'genres']
🎬 ¡Marchando! (ID: 7553) - Similitud: 2.264 - Razones: ['topic', 'genres']
🎬 Pasión sin barreras (ID: 41003) - Similitud: 2.202 - Razones: ['topic', 'genres']
🎬 London: Oscura obsesión (ID: 7515) - Similitud: 2.184 - Razones: ['topic', 'genres']
🎬 Academia antivicio 2 (ID: 28496) - Similitud: 2.178 - Razones: ['topic', 'genres']
